In [1]:
import CV_utils as cvu
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
import multiprocessing
%matplotlib inline

In [ ]:
def fourier_child(return_dict, i, video_frames, patch_size, verbose):

    video_frames = cvu.extract_patches_mean(video_frames, patch_size)
    
    # row: pixels_patches, col:time
    pixels_signal = np.zeros((len(video_frames[0]), len(video_frames)))
    
    for j, frame in enumerate(video_frames): pixels_signal[:,j] = frame

    # single video evolution
    pixels_transformed = []

    if verbose:
        start = time.time()
        print('Computing the Fourier serie; {}'.format(i+1))

    # analize row of pixels_signal: evolution of a patch in time
    for pixel_signal in pixels_signal:           
        # return the armonics' coefficients for the selected pixel
        coef = np.fft.rfft(pixel_signal)
        coef_r = coef.real
        coef_i = coef.imag

        coef = np.zeros(len(coef)*2)

        for k in range(len(coef_r)):
            coef[k*2] = coef_r[k]
            coef[(k*2)+1] = coef_i[k]

        pixels_transformed.append(coef)

    if verbose: print('Complete. Time spent: %s'%(time.time()-start))
        
    return_dict[i] = np.array(pixels_transformed).flatten()


In [ ]:
def fourier_dataset_creation(list_video_frames, patch_size, verbose=False):
    
    # result list: matrix list (one for each video) where row:#pixels, col:n_frames (time)
    pixels_transformed_videoList = []
    
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    jobs = []
           
    for i, video_frames in enumerate(list_video_frames):
                
        process = multiprocessing.Process(target=fourier_child, args=[return_dict, i, video_frames, patch_size, verbose])
        jobs.append(process)
        process.start()
            
    for p in jobs:
        p.join()
        
    for i in range(len(jobs)):    
        pixels_transformed_videoList.append(return_dict[i])
        
    return np.asmatrix(pixels_transformed_videoList)
    

In [ ]:
def load_fourier(path_to_read_binary, path_to_read_density, path_to_write, width, n_frames, patch_size=40, verbose=False):

    indexes_deleted = []
    
    for j in range(1, 16):
        
        if verbose: print("Extracting data for the videos from {} to {}".format((j-1)*10+1, j*10))

        list_binary_videoFrames = cvu.binary_read(path_to_read_binary, j, verbose=verbose)
        
        pd_densityFrames = pd.read_pickle(path_to_read_density+str(j)+'.pkl')

        list_frameList, indexes = cvu.normalize_preprocessing(list_binary_videoFrames, pd_densityFrames, width, n_frames, verbose=verbose)     

        X = fourier_dataset_creation(list_frameList, patch_size, verbose=verbose)
        
        bDel_indexes = [x+(j-1)*10 for x in indexes]
        indexes_deleted += bDel_indexes
        
        if verbose: print('\nWriting .csv file')          
        pd.DataFrame(X).to_csv(path_to_write+str(j)+".csv", index=False)
        
    return indexes_deleted

In [ ]:
indexes_deleted = load_fourier('Video/binary_video/', 'Video/densityFrame_pkl/videos_density_', 'datasets/pixels_fourier_partial/fourier_dataset_', width=680, n_frames=500, verbose=True)

#pd.DataFrame(indexes_deleted).to_csv("datasets/fourier_indexes_deleted.csv")

In [ ]:
#cvu.merge_dataset('datasets/pixels_fourier_partial/fourier_dataset_', 'datasets/fourier_dataset.csv', verbose=True)

## TESTING

In [2]:
X = pd.read_csv("datasets/fourier_dataset.csv").values

In [3]:
names, y = cvu.load_marks()
iDel = pd.read_csv('datasets/fourier_indexes_deleted.csv', index_col=0).values.flatten()
y = np.delete(y.values, iDel, axis=0)
y_l = y[:,0]
y_f = y[:,1]

In [7]:
X_tr, y_l_tr, y_f_tr, X_ts, y_l_ts, y_f_ts = cvu.random_sampling(X, y_l, y_f)

RLS

In [8]:
alphas = np.arange(0.001, 1, 0.007)

In [9]:
# LIGHTNESS
cvu.print_results('RLS',cvu.rlsCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts))

Model: RLS
	regularizer alpha: 0.995
	mean error: 0.6151352850422325
	variance of the error: 0.17134997140081168
	coefficient w:
[5.48408331e-09 0.00000000e+00 5.48393349e-09 ... 2.19259168e-10
 1.98223636e-09 0.00000000e+00]
	prediction:
[1.99673861 1.40520458 2.73225639 2.56059856 0.78684081 1.8536377
 2.1012893  1.67969287 1.49005283 1.25816831 2.27479114 2.92507966
 1.60821458 1.272312   0.96719124 2.304324   1.90788351 1.73377521
 2.45816299 1.89071167 2.12352753 1.70932074 2.52937584 1.55490044]


In [10]:
# FRAGILITY
cvu.print_results('RLS', cvu.rlsCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts))

Model: RLS
	regularizer alpha: 0.995
	mean error: 0.9313578320247394
	variance of the error: 0.3434813251163898
	coefficient w:
[-4.24059449e-09  0.00000000e+00 -4.24052150e-09 ... -1.06436990e-10
 -2.47042003e-09  0.00000000e+00]
	prediction:
[ 1.93985477  1.12983665  0.49038926  0.9424313   0.9944312   1.32176092
  1.46564475  1.13914488  1.71568833  1.95560436  2.24507844 -0.21704461
  1.94924862  1.72028612  1.22996868  0.99430375  0.72348263  1.810745
  1.61123787  1.38773957  0.4461687   2.17362943  0.8654023   1.24815522]


LASSO

In [11]:
alphas = np.arange(0.001, 1, 0.007)

In [12]:
#LIGHTNESS
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Model: LASSO
	regularizer alpha: 0.9600000000000001
	mean error: 0.5415756184342971
	variance of the error: 0.2295261193635474
	coefficient w:
[0. 0. 0. ... 0. 0. 0.]
	prediction:
[2.06269104 1.7948488  3.14027417 2.65899275 0.77165554 1.5168356
 1.38529144 1.88406153 1.29126947 0.9037815  3.04395411 3.67111924
 1.75155603 0.93129047 0.66520513 2.02266058 1.72090045 0.92360016
 1.63844268 2.37547715 2.30542584 1.54867518 2.6052379  1.38605493]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [13]:
#FRAGILITY
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Model: LASSO
	regularizer alpha: 0.988
	mean error: 1.013110492900998
	variance of the error: 0.4208902252400419
	coefficient w:
[-0.  0. -0. ... -0. -0.  0.]
	prediction:
[ 1.53655603  0.36252796  0.20911419  1.04767779  1.15641155  1.77625609
  1.99683809  1.25383035  2.6690327   2.20773367  2.63752635 -0.32955968
  2.77839337  1.37266842  1.13574301  0.24355087  0.65756433  1.66996654
  0.94730575  1.46714439 -0.36192091  1.59331629  0.89745383  0.98748603]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


KERNEL

In [14]:
alphas = {'alpha': list(np.arange(0.01, 20, 0.7))}
k1 = 'poly'
k2 = 'sigmoid'

In [15]:
#LIGHTNESS - poly
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts, kernel=k1))

Model: RIDGE KERNEL
	regularizer alpha: 0.01
	mean error: 0.8208494860398513
	variance of the error: 0.4244195371605799
	coefficient w:
[ 3.21610462e-19  5.33135207e-19  3.18639277e-18 -2.69277828e-19
  2.38219298e-19  2.06896050e-19  3.55554924e-19  2.39961579e-19
  1.99428186e-18  4.69405203e-19 -9.58300143e-19 -5.58129473e-19
 -3.10199615e-19  5.23145133e-19 -4.15464196e-19  2.26647071e-19
 -5.82985956e-19  1.32421669e-18 -3.31739143e-19  2.80856011e-19
 -5.83973244e-19 -2.35146981e-21  4.61876915e-19  4.31920161e-19
  1.30796869e-18  7.88270261e-19  5.75788195e-20 -3.91406162e-19
 -6.21374091e-19  2.47422437e-19  3.28873609e-19 -6.46358273e-19
 -3.80525971e-19  8.98613655e-19  5.55426939e-19 -1.44207615e-19
  1.03046390e-18  2.16189837e-18  1.07617708e-18 -3.60151320e-19
 -5.13565827e-19  1.83266511e-19  4.46864456e-19  5.07786923e-19
  1.41562696e-18  2.78812795e-18 -5.77836085e-19 -5.35218880e-19
  1.88593698e-19  3.02047930e-19 -6.03152918e-19  1.73450399e-18
 -3.95203146e-20 -1

In [16]:
#LIGHTNESS - sigmoid
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts, kernel=k2))

Model: RIDGE KERNEL
	regularizer alpha: 0.71
	mean error: 0.6833333333333335
	variance of the error: 0.2454281846625818
	coefficient w:
[-0.29491615 -0.29491615  1.39522469 -1.70336686  1.11353455  1.11353455
 -1.42167672 -1.70336686  0.83184441 -0.29491615 -1.985057   -1.42167672
 -0.29491615  0.55015427  0.26846413  1.67691483 -0.01322601 -1.42167672
  0.26846413  0.83184441 -1.13998657 -0.29491615  1.39522469  0.26846413
  0.26846413 -0.01322601  0.55015427 -0.57660629 -1.70336686 -0.29491615
  1.67691483 -1.13998657 -1.13998657  2.52198526  0.26846413 -0.85829643
  0.55015427 -1.13998657  1.39522469 -0.85829643 -0.85829643 -1.42167672
 -0.57660629 -0.57660629 -0.29491615  2.52198526 -1.70336686 -0.57660629
 -1.13998657  0.26846413 -1.42167672  2.52198526  0.55015427  0.83184441
  0.83184441  0.55015427 -0.29491615  2.24029512 -0.29491615 -1.13998657
 -0.29491615  1.67691483  1.95860497 -0.85829643  0.83184441 -1.985057
  0.55015427 -0.01322601  2.24029512  0.55015427]
	prediction:


In [17]:
#FRAGILITY - poly
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts, kernel=k1))

Model: RIDGE KERNEL
	regularizer alpha: 16.11
	mean error: 1.040696484846188
	variance of the error: 0.37262328068961015
	coefficient w:
[-2.20402495e-19  1.64651516e-18 -4.20947779e-19  6.26498715e-19
  3.11532165e-19 -6.67716231e-19  8.67151394e-19  7.13354511e-19
 -2.13749058e-19  1.35807935e-18 -6.89307014e-20 -3.89048077e-19
 -2.39635169e-19  6.40545444e-19 -9.94470397e-20 -2.97370050e-19
 -9.32361463e-19  6.05260006e-18  3.80200667e-19 -2.39021593e-19
 -1.89653968e-19  8.66233186e-19 -1.05552602e-18  4.78986663e-19
  1.02951521e-18 -3.27949115e-19 -1.05538024e-18 -4.90562148e-20
  6.78604028e-19 -7.43891983e-19 -2.64147043e-19  5.39490963e-19
  6.00545385e-19 -1.24984672e-19  3.77071371e-19 -3.77605446e-19
  2.45046359e-18  1.05813507e-17 -1.36179139e-19 -6.23692510e-19
  3.10727066e-19  6.55226078e-19 -2.89045085e-19  2.06078839e-18
  9.94448483e-19 -6.96111798e-19 -1.26409683e-19 -2.76871449e-19
  1.55211978e-18  1.87696780e-18 -1.77603613e-19 -5.58411149e-19
  6.20032209e-19 -

In [18]:
#FRAGILITY - sigmoid
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts, kernel=k2))

Model: RIDGE KERNEL
	regularizer alpha: 1.41
	mean error: 0.9726964010642768
	variance of the error: 0.268812712405795
	coefficient w:
[-0.54777079  0.30329304 -0.83145873  0.72882496 -0.12223887 -0.9733027
 -0.12223887  0.0196051  -0.68961476  0.44513701 -0.26408285 -0.68961476
  0.16144907  0.30329304  1.0125129  -0.54777079 -0.40592682  0.72882496
  1.15435687 -0.54777079 -0.54777079  1.0125129  -0.83145873  0.16144907
  1.43804481 -0.9733027  -0.83145873  0.72882496  0.58698098 -0.68961476
 -0.68961476 -0.12223887  0.16144907 -0.54777079 -0.12223887 -0.68961476
  1.43804481  1.15435687 -0.40592682 -0.54777079 -0.12223887  0.0196051
  0.16144907  0.30329304 -0.26408285 -0.68961476 -0.40592682  0.16144907
  0.87066893  1.43804481 -0.68961476 -0.83145873  1.0125129   0.30329304
  1.0125129   1.0125129   0.87066893 -0.83145873 -0.40592682  0.44513701
  0.30329304 -0.83145873 -0.68961476 -0.26408285  0.87066893 -0.26408285
 -0.68961476  0.72882496 -0.9733027   1.0125129 ]
	prediction:
[

SVM

In [19]:
c={'C':np.arange(1,100,10)}

In [20]:
#LIGHTNESS
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, y_l_tr, y_l_ts))

Model: SVM
	regularizer alpha: 11
	mean error: 0.6833333333333335
	variance of the error: 0.23607775797628838
	coefficient w:
[[-0.13125    -0.13125     0.86796875 -1.13125     0.66875     0.66875
  -0.93125    -1.13125     0.46875    -0.13125    -1.33125    -0.93125
  -0.13125     0.26875     0.06875     1.06875    -0.93125     0.06875
   0.46875    -0.73125    -0.13125     0.86875     0.06875     0.06875
   0.26875    -0.33125    -1.13125    -0.13125     1.06875    -0.73125
  -0.73125     1.66875     0.06875    -0.53125     0.26875    -0.73125
   0.86875    -0.53125    -0.53125    -0.93125    -0.33125    -0.33125
  -0.13125     1.66875    -1.13125    -0.33125    -0.73125     0.06875
  -0.93203125  1.66875     0.26875     0.46875     0.46875     0.26875
  -0.13203125  1.46875    -0.13203125 -0.73203125 -0.13203125  1.06875
   1.26875    -0.53203125  0.46875    -1.33203125  0.26875     1.46875
   0.26875   ]]
	prediction:
[1.63134328 1.63134328 1.63134328 1.63134328 1.63134328 1.631343

In [21]:
#FRAGILITY
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, y_f_tr, y_f_ts))

Model: SVM
	regularizer alpha: 11
	mean error: 0.9764705882352939
	variance of the error: 0.2662399077277972
	coefficient w:
[[-0.70859375  0.29140625 -1.10859375  0.89140625 -0.10859375 -1.30859375
  -0.10859375 -0.90859375  0.49140625 -0.30859375 -0.90859375  0.09140625
   0.29140625  1.290625   -0.70859375 -0.50859375  0.89140625  1.49140625
  -0.70859375 -0.70859375  1.290625   -1.10859375  0.09140625  1.89140625
  -1.30859375 -1.10859375  0.89140625  0.69140625 -0.90859375 -0.90859375
  -0.10859375  0.09140625 -0.70859375 -0.109375   -0.909375    1.89140625
   1.49140625 -0.509375   -0.709375   -0.109375    0.09140625  0.29140625
  -0.309375   -0.909375   -0.509375    0.09140625  1.09140625  1.89140625
  -0.909375   -1.109375    1.29140625  0.29140625  1.29140625  1.29140625
   1.09140625 -1.109375   -0.509375    0.49140625  0.29140625 -1.109375
  -0.909375   -0.309375    1.09140625 -0.309375   -0.909375    0.89140625
  -1.309375    1.29140625]]
	prediction:
[1.40882353 1.40882353

RF

In [22]:
#LIGHTNESS
ris = cvu.tree_regression(X_tr, X_ts, y_l_tr, y_l_ts)
print(ris[0], ris[1])
print(ris[2])

0.8416666666666668 0.7499305555555553
[0. 0. 0. ... 0. 0. 0.]


In [23]:
#FRAGILITY
ris = cvu.tree_regression(X_tr, X_ts, y_f_tr, y_f_ts)
print(ris[0], ris[1])
print(ris[2])

1.1916666666666667 0.8749305555555557
[0. 0. 0. ... 0. 0. 0.]


GROUP LASSO

In [25]:
groups = np.array(range(0, (306*502))).reshape(-1,2).tolist()
X_tr, Y_l_tr, Y_f_tr, X_ts, Y_l_ts, Y_f_ts = cvu.random_sampling(X, y_l, y_f)
alphas = np.arange(0.001, 1, 0.01)

In [28]:
# LIGHTNESS
coef, alpha = cvu.group_lassoCV(X_tr, Y_l_tr, alphas, groups, max_iter=cvu.MAX_ITER, rtol=1e-6)
res = abs(Y_l_ts-np.dot(X_ts,coef))
err = np.mean(res)
var = np.var(res)
print(err)
print(var)
print(coef)

1.675
0.7260416666666668
[ 0.  0.  0. ...  0. -0.  0.]


In [29]:
# FRAGILITY
coef, alpha = cvu.group_lassoCV(X_tr, Y_f_tr, alphas, groups, max_iter=cvu.MAX_ITER, rtol=1e-6)
res = abs(Y_l_ts-np.dot(X_ts,coef))
err = np.mean(res)
var = np.var(res)
print(err)
print(var)
print(coef)

/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep u

/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep u

/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep u

/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep u

/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  L, A.dot(q))[0])[0]) / rho
/Users/danilo/Desktop/projCV/CV_utils.py:533: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep u

1.675
0.7260416666666668
[ 0.  0.  0. ... -0. -0.  0.]
